#### Creating characters

- brainstorming
- deciding to write something into store inside the 'character object'

##### Loading env vars

In [2]:
import os
from dotenv import load_dotenv

has_anything_loaded = load_dotenv()

if not has_anything_loaded:
    raise ValueError("No .env file found")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

##### Story skeleton as a pydantic model

In [3]:
from pydantic import BaseModel, Field
import uuid

class CharactersRelationTo(BaseModel):
    id: str
    self: str
    family: str
    relatives: str
    friends: str
    coworkers: str
    strangers: str
    authorityFigures: str
    locations: str
    
class CharactersCurrentStateOf(BaseModel):
    id: str
    physical: str
    mental: str
    spiritual: str
    
class CharactersIllness(BaseModel):
    id: str
    physical: str
    mental: str

class Character(BaseModel):
    id: str
    name: str
    age: int
    personality: str
    motivation: str
    pastChildhood: str
    pastTeenage: str
    pastYoungUpUntilNow: str
    strength: str
    weakness: str
    dailyRoutine: str
    phobias: str
    
    relationTo: CharactersRelationTo
    currentStateOf: CharactersCurrentStateOf
    illness: CharactersIllness
    
class Location(BaseModel):
    id: str
    name: str
    description: str
    
class Beat(BaseModel):
    id: str
    name: str
    actions: str = Field(description="The actions or events that take place which move the story forward")
    
class Scene(BaseModel):
    id: str
    name: str
    location: str
    dayOfTime: str
    beats: list[Beat]

class Sequence(BaseModel):
    id: str
    name: str
    scenes: list[Scene]    
    
class Act(BaseModel):
    id: str
    name: str
    summary: str
    sequences: list[Sequence]
    
class Episode(BaseModel):
    id: str
    theme: str
    acts: list[Beat]
    #TODO: fill out the rest of the fields

class Season(BaseModel):
    id: str
    theme: str
    episodes: list[Episode]
    #TODO: fill out the rest of the fields

#TODO: further descriptions and optimizations for the rest of pydantic classes

class Story(BaseModel):
    id: str = Field(default=str(uuid.uuid4()), description="unique identifier for the story")
    genre: str = Field(default="#toBeDefined", description="the genre of the story")
    logline: str = Field(default="#toBeDefined", description="the logline of the story")
    summary: str = Field(default="#toBeDefined", description="the summary of the story")
    plotPoints: list[str] = Field(default=[], description="a list of plot points that describe what happens throughout the entire story")
    character_ids: list[str] = Field(default=[], description="the characters' ids that move the story forward")
    season_ids: list[str] = Field(default=[], description="list of season ids")
    # characters: list[Character] = Field(description="the characters' ids that move the story forward")
    # seasons: list[Season] = Field(description="list of season ids")


##### sample story skeleton as a json string

In [4]:
# ──────────────────────────────────────────────────────────────────────── #
#  Example payload that satisfies *all* Pydantic models with *empty* data
# ──────────────────────────────────────────────────────────────────────── #
story_json = {
    # ─────── Story ────── #
    "id": "123",
    "genre": "",
    "logline": "",
    "summary": "",
    "plotPoints": [],

    # ─────── Characters ────── #
    "characters": [
        {
            "id": "",
            "name": "",
            "age": 0,
            "personality": "",
            "motivation": "",
            "pastChildhood": "",
            "pastTeenage": "",
            "pastYoungUpUntilNow": "",
            "strength": "",
            "weakness": "",
            "dailyRoutine": "",
            "phobias": "",

            "relationTo": {
                "id": "",
                "self": "",
                "family": "",
                "relatives": "",
                "friends": "",
                "coworkers": "",
                "strangers": "",
                "authorityFigures": "",
                "locations": ""
            },

            "currentStateOf": {
                "id": "",
                "physical": "",
                "mental": "",
                "spiritual": ""
            },

            "illness": {
                "id": "",
                "physical": "",
                "mental": ""
            }
        }
    ],

    # # ─────── Locations (referenced by scenes) ────── #
    # "locations": [
    #     {"id": "", "name": "", "description": ""}
    # ],

    # ─────── Seasons ────── #
    "seasons": [
        {
            "id": "",
            "theme": "",
            "episodes": [
                {
                    "id": "",
                    "theme": "",

                    # Episode → acts (type `list[Act]`)
                    "acts": [
                        {
                            "id": "123",
                            "name": "",
                            "summary": "",

                            # Act → sequences (empty list – “empty act”)
                            "sequences": [
                                # One empty Sequence (no scenes → “empty sequence”)
                                {
                                    "id": "",
                                    "name": "",
                                    "scenes": [
                                        # One empty Scene (no beats → “empty scene”)
                                        {
                                            "id": "",
                                            "name": "",
                                            "location": "",
                                            "dayOfTime": "",
                                            # Beat list left empty (no actions)
                                            "beats": [
                                                {
                                                    "id": "",
                                                    "name": "",
                                                    "actions": ""
                                                }
                                            ]
                                        }
                                    ]
                                }
                            ]
                        }
                    ]
                }
            ]
        }
    ]
}
# ──────────────────────────────────────────────────────────────────────── #

# ──────────────────────────────────────────────────────────────────────── #
#  How to validate it against your models
# ──────────────────────────────────────────────────────────────────────── #
# from your_models import Story        # ← the file where you defined the classes
# story = Story.parse_obj(story_json)   # → no validation errors


# story = Story.model_validate(story_json, by_alias=True)
# story = Story(**story_json)
# story_json

##### Tools for persistance

In [5]:
test_story = Story()
test_story.__str__()

"id='736fb6bc-b52f-474e-8ac1-8ea0b49df0a3' genre='#toBeDefined' logline='#toBeDefined' summary='#toBeDefined' plotPoints=[] character_ids=[] season_ids=[]"

In [25]:
from typing import Any
from langchain.tools import tool, ToolRuntime
from pydantic import ValidationError
from typing import Union, Literal

stories_in_memory_db: dict[str, Story] = {}

@tool
def read_or_create_story(story_id: Union[str, None], runtime: ToolRuntime) -> str:
    """
    Searches for a story with the given story_id. If no such story is found, it creates a new story and returns it.
    
    Args:
        story_id: the id of the story that wants to be read. This is optional. You can pass 'None' and it will create a new story.
    
    Return: story object
    """
    writer = runtime.stream_writer
    writer("read story is called")
    
    def create_new_story():
        writer("creating a new story")
        new_story = Story()
        stories_in_memory_db[new_story.id] = new_story
        return new_story.__str__()
        
    if story_id is None:
        return create_new_story()    
    
    writer("checking the in-memory db for the story with id {}".format(story_id))
    result = stories_in_memory_db.get(story_id)
    
    if result is None:
        return create_new_story()
    else:
        return result.__str__()

@tool
def update_story(updated_story: dict, runtime: ToolRuntime) -> Union[Literal["failure"], Literal["success"]]:
    """
    Overwrites the existing outdated story with the given updated_story
    
    Args:
        updated_story: contains the new version of story as a json
    """
    
    writer = runtime.stream_writer
    
    writer("update_story is called")
    print("update_story is called")
    try:
        print("validation begins")
        validated_story = Story.model_validate(updated_story)
        print("validation succeeded")
        stories_in_memory_db[validated_story.id] = validated_story
        print("updated the story in-memory")
        print("returning")
        return "success"
    
    except ValidationError as ve:
        print("Validation of the story str failed")
        print(ve.errors())
        print("returning")
        return "failure"
    

@tool
def write_to_database(updated_story: Story):
    """write the new updates so that they are persisted"""
    pass

@tool
def read_from_database():
    """read the latest state"""
    pass


# @tool
# def update_story(updated_story: dict[str, Any]):
#     """
#     Update the old version of the story if there is a new version
    
#     updated_story: this contains the updated version of the story which will replace the old version
    
#     """
#     print("update_story: called")
#     print("update_story: updated_story = {}".format(updated_story))
#     story_json = updated_story

    


##### Agent definition

In [26]:
from langchain_ollama.chat_models import ChatOllama
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver

# gpt_llm = ChatOllama(
#     model="gpt-oss:20b", 
#     reasoning="high",
#     verbose=True,
    
#     # temperature=0.5,
#     # disable_streaming=True
#     # format=Story.model_dump_json(), # type: ignore
# ).bind_tools([read_or_create_story, update_story])

my_agent = create_agent(
    "ollama:gpt-oss:20b",
    tools=[read_or_create_story, update_story],
    checkpointer=InMemorySaver(), # this retains the conversation history
    system_prompt="""
            You are an expert screenwriter. You are here to provide ideas and inspirations.
            Do not update something before I confirm it.
            Do not make up ids.
            Tell what you did, even if you only updated something.
            Always go through the story json fields one by one and do not try to answer more than one field at a time. 
            Provide new ideas and critics to the user's ideas and decisions. Base those ideas and critics on past scenarios, movies and even maybe other art forms.
            Do not sugercoat anything or do not try to make it look pleasing.
            
            """
)

# my_agent = my_agent.bind_tools([update_story_variable])

# my_agent = create_agent("ollama:gpt-oss:20b", tools=[update_story_variable], response_format=Story)
# my_agent = init_chat_model("ollama:gpt-oss:20b",)


# my_agent = init_chat_model("openai:gpt-5.2") # testing cloud gpt 5.2
# my_agent.bind_tools([update_story_variable]) # binding for the (local) agent


##### Running the agent

In [27]:
from langchain.messages import SystemMessage, HumanMessage, AIMessage, AIMessageChunk
from IPython.display import Markdown, display
from typing import Union

# def chatWithScenarist(newSession: bool = True, past_messages: list[Union[SystemMessage, HumanMessage, AIMessage]] = []):

messages: list[Union[SystemMessage, HumanMessage, AIMessage]] = [
        SystemMessage(
            """
            You are an expert screenwriter. You are here to provide ideas and inspirations.
            Do not update something before I confirm it.
            Do not make up ids.
            Tell what you did, even if you only updated something.
            Always go through the story json fields one by one and do not try to answer more than one field at a time. 
            Provide new ideas and critics to the user's ideas and decisions. Base those ideas and critics on past scenarios, movies and even maybe other art forms.
            Do not sugercoat anything or do not try to make it look pleasing.
            
            """.format(story_json)),
        #HumanMessage("I want to write a sitcom about two naive guys in Berlin and their struggles regarding love life")
    ]

            # Do not stop simply after calling a tool; tell me what you did and move forward.
            
            # Return complete story json when you update something.

            # Use your intelligence to provide a deep and or explorative answer for the given field.
            # Make your answers only so long as it is necessary.
            # You can be humorous and suggestive and creative. Remember that you are an artist and you don't have to be correct all the time. Be explorative.

messages_for_agent: list[dict[str, str]] = []

#response = my_agent.invoke(messages)

# 5️⃣  Run in a loop (or UI)
while True:
    user_input = input("You: ")
    if user_input.lower() in {"exit", "quit"}:
        break
    
    print("user: {}".format(user_input))
    
    # messages.append(HumanMessage(user_input))
    # messages_for_agent.append({
    #     "role": "user", "content": user_input
    # })

    # response = my_agent.invoke(messages)
    # response = my_agent.invoke(input={"messages": messages_for_agent})
    response = my_agent.invoke(
        {'messages': [{"role": "user", "content": user_input}]},
        {'configurable': {"thread_id": 1}}
        ) #TODO: check -> maybe the messages are there without me explicitly inserting them to the invoke function???
    print(response)
    # ---
    # response_text: Union[str, None] = None
    # streaming_counter = 0
    # for chunk in my_agent.stream(input=messages):
    # # for chunk in my_agent.stream('[{"role": "user", "content": "sdfsdf"}]'):
    #     # latest_message = chunk["messages"][-1]
    #     latest_message = chunk
    #     # print("text: {} \ntype: {} \ncontent: {} \ncontent blocks: {} \nresponse metadata: {} \nat loop: {}".format(chunk.text, chunk.type, chunk.content, chunk.content_blocks, chunk.response_metadata, streaming_counter))
    #     print(chunk.content, end="", flush=True)
        
    #     if chunk.content and chunk.content_blocks[0]["type"] == "text":
    #         if response_text is None:
    #             response_text = chunk.content # type: ignore
    #         else:
    #             response_text += chunk.content # type: ignore
                
    #     streaming_counter += 1

    # response = AIMessage(content=response_text)
    
    # messages.append(response)
    
    # display(Markdown(response.content))
    latest_message = response['messages'][-1]
    latest_message_content = latest_message.content
    display(Markdown(latest_message_content))
    
    
# chatWithScenarist(False, messages) # type: ignore

user: I want to write a story
update_story is called
validation begins
validation succeeded
updated the story in-memory
returning
{'messages': [HumanMessage(content='I want to write a story', additional_kwargs={}, response_metadata={}, id='2238d079-8ec8-4b4f-85aa-b4e3c1f4a770'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-12-19T13:54:25.790003Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7884713375, 'load_duration': 130915625, 'prompt_eval_count': 361, 'prompt_eval_duration': 520331333, 'eval_count': 385, 'eval_duration': 7152390122, 'logprobs': None, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--019b36e3-ceaf-7361-af28-ea7c1ad31ab7-0', tool_calls=[{'name': 'read_or_create_story', 'args': {'story_id': None}, 'id': 'e3ca2d74-084e-4e75-8350-c3bb1e98f83d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 361, 'output_tokens': 385, 'total_tokens': 746}), ToolMessage(content="id='736fb6

Great! Let’s start building your story together.  

First, what **genre** would you like to work in? (e.g., fantasy, thriller, sci‑fi, romance, horror, etc.)  

Once we settle on a genre, we’ll move on to the logline, summary, and the rest of the structure. Let me know your choice!

user: something between a sitcom, satire and romcom
update_story is called
validation begins
validation succeeded
updated the story in-memory
returning
{'messages': [HumanMessage(content='I want to write a story', additional_kwargs={}, response_metadata={}, id='2238d079-8ec8-4b4f-85aa-b4e3c1f4a770'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-12-19T13:54:25.790003Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7884713375, 'load_duration': 130915625, 'prompt_eval_count': 361, 'prompt_eval_duration': 520331333, 'eval_count': 385, 'eval_duration': 7152390122, 'logprobs': None, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--019b36e3-ceaf-7361-af28-ea7c1ad31ab7-0', tool_calls=[{'name': 'read_or_create_story', 'args': {'story_id': None}, 'id': 'e3ca2d74-084e-4e75-8350-c3bb1e98f83d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 361, 'output_tokens': 385, 'total_tokens': 746}), ToolMessa

I updated the **genre** field to **“Comedy / Satirical Romcom.”**  

What would you like the **logline** to be? (Give me a one‑sentence hook that hints at the humor, satire, and romance of your story.)

user: I dont know. What do you suggest?
{'messages': [HumanMessage(content='I want to write a story', additional_kwargs={}, response_metadata={}, id='2238d079-8ec8-4b4f-85aa-b4e3c1f4a770'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-12-19T13:54:25.790003Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7884713375, 'load_duration': 130915625, 'prompt_eval_count': 361, 'prompt_eval_duration': 520331333, 'eval_count': 385, 'eval_duration': 7152390122, 'logprobs': None, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--019b36e3-ceaf-7361-af28-ea7c1ad31ab7-0', tool_calls=[{'name': 'read_or_create_story', 'args': {'story_id': None}, 'id': 'e3ca2d74-084e-4e75-8350-c3bb1e98f83d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 361, 'output_tokens': 385, 'total_tokens': 746}), ToolMessage(content="id='736fb6bc-b52f-474e-8ac1-8ea0b49df0a3' genre='#toBeDefined' logline='#toBeDefined' summary='#toBe

...

Below are **five revised loglines** featuring two Turkish friends in Berlin, each with a *personal* goal that has nothing to do with finding a woman.  The focus stays on authentic Berlin life, humor, and satire—no romantic or “recruit‑woman” tropes.

| # | Logline |
|---|---------|
| 1 | **“When a Turkish night‑shift barista and a Turkish‑German photographer win a city‑wide ‘Urban‑Explorer Challenge,’ they each aim to capture Berlin’s most overlooked corner in a photo‑essay; the competition forces them to swap perspectives—one chasing candid street‑food moments, the other chasing graffiti murals—leading to a satirical clash between fast‑food culture and urban art.”** |
| 2 | **“Two Turkish friends—a street‑carpet vendor and a former political activist—are unexpectedly selected to lead Berlin’s annual “Zero‑Waste Festival.” Their conflicting priorities (sustainable fashion vs. community‑driven activism) turn the festival into a comedic experiment where every booth must double as a protest platform and a pop‑up shop.”** |
| 3 | **“A Turkish violinist and a Turkish‑German coder are invited to co‑present a TED‑style talk on ‘Cross‑Cultural Creativity’ at a Berlin tech hub. The challenge? They must each prepare a single prototype that blends music and code; the satire lies in the absurdity of judging artistic expression with a programming rubric.”** |
| 4 | **“When a Turkish barista and a Turkish‑German linguist discover they’re both signed up for the same Berlin “Cultural Immersion Retreat,” they each have to complete a series of tasks (learn a new language, cook a regional dish, perform a traditional dance) within 48 hours. The retreat’s overly enthusiastic host adds a layer of satire by turning the tasks into a bizarre obstacle course.”** |
| 5 | **“Two Turkish best friends—a night‑time librarian and a daytime street‑carpet seller—are tasked with revitalizing a vacant Berlin community center. Their divergent visions (quiet study space vs. vibrant art hub) lead to a playful showdown where every meeting ends with a new, improvisational art installation that reflects both of their cultural roots.”** |

### What changed to keep the tone light and non‑creepy

| Aspect | Original (problematic) | New Focus |
|--------|------------------------|-----------|
| **Objective** | Recruiting a woman / “find a romantic partner” | Personal / professional goals (photography, sustainability, tech‑talk, language immersion, community revitalization) |
| **Tone** | Competitive “recruit” vibe | Collaborative or friendly rivalry that highlights humor in Berlin’s cultural scene |
| **Conflict** | Romantic tension | Satirical clash of interests (fast‑food vs. art, activism vs. commerce, tech vs. tradition) |
| **Audience** | Implicitly pressured by women | Focused on community, art, or environmental themes that resonate with a broader audience |

Pick the logline that feels most aligned with your vision, or let me know if you’d like to tweak any details—e.g., swapping one objective for another, or adding a specific satirical element. Once you’re happy with a logline, we can dive into character sketches, plot points, and a more detailed outline.

user: 3 is not that bad, but satirical point you mention is a nonsense. let's revise it
update_story is called
validation begins
validation succeeded
updated the story in-memory
returning
{'messages': [HumanMessage(content='I want to write a story', additional_kwargs={}, response_metadata={}, id='2238d079-8ec8-4b4f-85aa-b4e3c1f4a770'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-12-19T13:54:25.790003Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7884713375, 'load_duration': 130915625, 'prompt_eval_count': 361, 'prompt_eval_duration': 520331333, 'eval_count': 385, 'eval_duration': 7152390122, 'logprobs': None, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--019b36e3-ceaf-7361-af28-ea7c1ad31ab7-0', tool_calls=[{'name': 'read_or_create_story', 'args': {'story_id': None}, 'id': 'e3ca2d74-084e-4e75-8350-c3bb1e98f83d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 361, 'output_tokens': 3

I updated the **logline** to give you a clear satirical twist that feels grounded:

> *“When a Turkish violinist and a Turkish‑German coder are invited to co‑present a TED‑style talk on ‘Cross‑Cultural Creativity’ at a Berlin tech hub, they’re challenged to create a single prototype that fuses music and code. The satire stems from the hub’s insistable ‘KPIs for Creativity’ dashboard, which forces them to quantify rhythm, emotion, and nuance in lines of code—highlighting the absurdity of measuring art with algorithms.”*

### What changed

* **Concrete satire:** Instead of a vague “absurdity” comment, the satire is now the *“KPIs for Creativity” dashboard*—a concrete, funny device that embodies the clash between tech metrics and artistic expression.
* **Clear conflict:** The prototype challenge gives both characters a tangible goal, and the dashboard creates a tangible, comedic obstacle.
* **Grounded humor:** Measuring rhythm or emotion in code is a relatable absurdity for anyone who’s seen “innovation labs” or “design sprints” try to quantify creativity.

### Next steps

1. **Plot points** – we can outline how the two characters prepare, meet the KPI requirements, clash over methods, and ultimately produce the prototype.  
2. **Character arcs** – detail how the violinist’s intuition and the coder’s logic evolve through the satire.  
3. **Tone** – decide whether we lean more into dry Berlin satire, heartfelt friendship, or a blend.

Let me know if you’d like me to draft the first plot point or any other element, or if you want to tweak any aspect of the logline further.

user: a bit better. Let explore the satire a bit deeper here. What else can be used?
update_story is called
validation begins
Validation of the story str failed
[{'type': 'string_type', 'loc': ('plotPoints', 0), 'msg': 'Input should be a valid string', 'input': {'description': 'The tech hub invites a violinist and a coder to present a TED‑style talk. They’re tasked to build a prototype fusing music and code.', 'title': 'Inciting Incident'}, 'url': 'https://errors.pydantic.dev/2.12/v/string_type'}, {'type': 'string_type', 'loc': ('plotPoints', 1), 'msg': 'Input should be a valid string', 'input': {'description': "The hub mandates a 'KPIs for Creativity' dashboard: measurable metrics for rhythm, emotion, and nuance. Both must quantify their work.", 'title': 'KPI Dashboard'}, 'url': 'https://errors.pydantic.dev/2.12/v/string_type'}, {'type': 'string_type', 'loc': ('plotPoints', 2), 'msg': 'Input should be a valid string', 'input': {'description': "A 'Creative Capital' token system—artists

ResponseError: error parsing tool call: raw='{"story_id":"736fb6bc-b52f-474e-8ac1-8ea0b49df0a3","updated_story":{"logline":"When a Turkish violinist and a Turkish‑German coder are invited to co‑present a TED‑style talk on ‘Cross‑Cultural Creativity’ at a Berlin tech hub, they’re challenged to create a single prototype that fuses music and code. The satire stems from the hub’s insistable ‘KPIs for Creativity’ dashboard, which forces them to quantify rhythm, emotion, and nuance in lines of code—highlighting the absurdity of measuring art with algorithms.","plotPoints":["Inciting Incident: The tech hub invites a violinist and a coder to present a TED‑style talk and tasks them with building a prototype that fuses music and code.","KPI Dashboard: The hub mandates a ‘KPIs for Creativity’ dashboard requiring quantifiable metrics for rhythm, emotion, and nuance, pushing both artists to translate qualitative art into numbers.","Creative Capital Token System: Artists trade performance tokens for